In [1]:
import os
import uuid
from openai import OpenAI
from utils.file_manager import FileManager
from utils.conversation import Conversation
from utils.llm_manager import Assistant
from db.db_manager import MongoDBManager
from dotenv import load_dotenv


In [2]:
file_manager = FileManager()
conversation_memory_prompt = file_manager.load_md_file('prompts/conversation_memory.md')
sales_detector_prompt = file_manager.load_md_file('prompts/sales_detector.md')
consentiment_prompt = file_manager.load_md_file('prompts/consentiment.md')

load_dotenv()

api_key = os.environ.get("THE_OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

#Assistants
assistant_memory = Assistant(client=client, base_prompt=conversation_memory_prompt)
assistant_sales_detector = Assistant(client=client, base_prompt=sales_detector_prompt)
assistant_consentiment = Assistant(client=client, base_prompt=consentiment_prompt)

#MongoDB
db = MongoDBManager()

In [3]:
def format_var_chat_history(resultados: list[dict])-> str:
    """This function recievesa list of Conversations and for each row (dict), retrieves the questiona dn answer, and generate a string based on that values."""
    conversacion = ""
    
    for registro in resultados:
        question = registro.get("question")
        answer = registro.get("answer")
        
        if question:
            conversacion += f"\nUser: {question}"
        
        if answer:
            conversacion += f"\nAssistant: {answer}"
    
    return conversacion.strip()  # Remove any unnecessary line breaks at the beginning or end

In [4]:
db.buscar_conversaciones_por_id(conversation_id="67c2293c-73d9-46f6-8d7c-5de46d4f87a2")

[{'_id': ObjectId('66ef0c3d1013b8430764f78d'),
  'conversation_id': '67c2293c-73d9-46f6-8d7c-5de46d4f87a2',
  'question': 'Can I return an item?',
  'answer': 'The refund process takes 5-7 business days.',
  'sales_intention': False,
  'concent': False},
 {'_id': ObjectId('66ef0c3d1013b8430764f78e'),
  'conversation_id': '67c2293c-73d9-46f6-8d7c-5de46d4f87a2',
  'question': 'Can I track my order?',
  'answer': 'Yes, there is a 1-year warranty on all products.',
  'sales_intention': False,
  'concent': False}]

In [5]:
conversations = db.buscar_conversaciones_por_id(conversation_id="67c2293c-73d9-46f6-8d7c-5de46d4f87a2")

In [6]:
var_chat_history = format_var_chat_history(resultados=conversations)

In [7]:
var_chat_history

'User: Can I return an item?\nAssistant: The refund process takes 5-7 business days.\nUser: Can I track my order?\nAssistant: Yes, there is a 1-year warranty on all products.'

In [10]:
assistant_memory.base_prompt

'Given the following conversation and a follow-up question, \n- summarize the content of the conversation, by keeping **ONLY CONTENT RELEVANT TO THE QUESTION** in order to be understood\n- For questions in one of the following languages: English, French, German, Italian, Portuguese, or Spanish, you must write the summary in the same language of the question. For instance, a question in Spanish requires a summary in Spanish.\n- export the follow-up question along with the summary according to the following structure: \n\n<relevant_chat_history_synthesis>\nsummary of relevant chat history\n</relevant_chat_history_synthesis>\n\n<follow_up_input>\n- follow-up input\n</follow_up_input>\n\nFor the current input:\nChat History:\n{var_chat_history}\nFollow-Up Input: {question}\nOutput:'

In [17]:
print(assistant_memory.base_prompt.format(var_chat_history=var_chat_history,question="Can you repeat how many days?"))

Given the following conversation and a follow-up question, 
- summarize the content of the conversation, by keeping **ONLY CONTENT RELEVANT TO THE QUESTION** in order to be understood
- For questions in one of the following languages: English, French, German, Italian, Portuguese, or Spanish, you must write the summary in the same language of the question. For instance, a question in Spanish requires a summary in Spanish.
- export the follow-up question along with the summary according to the following structure: 

<relevant_chat_history_synthesis>
summary of relevant chat history
</relevant_chat_history_synthesis>

<follow_up_input>
- follow-up input
</follow_up_input>

For the current input:
Chat History:
User: Can I return an item?
Assistant: The refund process takes 5-7 business days.
User: Can I track my order?
Assistant: Yes, there is a 1-year warranty on all products.
Follow-Up Input: Can you repeat how many days?
Output:


In [18]:
#question debe ser ""
#format base_prompt {var_chat_history}: var_chat_history, {question}:question
test1 = assistant_memory.assistant_chat_completion_response(prompt=assistant_memory.base_prompt.format(var_chat_history=var_chat_history,question="Can you tell if I can tack my order?"), question="")

In [19]:
print(test1)

<relevant_chat_history_synthesis>
User: Can I track my order?
Assistant: Yes, there is a 1-year warranty on all products.
</relevant_chat_history_synthesis>

<follow_up_input>
- Can you tell if I can tack my order?
</follow_up_input>
